In [14]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.tools import tool
from langchain_core.runnables import RunnableLambda



load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)

# Example: Use Gemini
chain = prompt | gemini_llm
response = chain.invoke({"product": "smart home devices"})
print("Gemini response:", response)

# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)
import langchain
print("LangChain version:", langchain.__version__)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
Gemini response: content='Here are some creative names for a smart home device company, playing on different aspects like intelligence, comfort, connectivity, and the future:\n\n**Evocative & Modern:**\n\n1.  **AuraFlow:** Suggests seamless energy and a guiding presence.\n2.  **LumenHaus:** "Lumen" (light/illumination) + "Haus" (German for home) implies clarity and modern living.\n3.  **Synapse Home:** Evokes the neural connections in the brain, suggesting intelligence and connectivity.\n4.  **Veridian Living:** "Veridian" suggests growth, freshness, and sophisticated greenery, hinting at sustainable, smart living.\n5.  **Echelon Living:** Implies a higher level or standard of home automation.\n6.  **WispWire:** Suggests effortless, invisible connectivity and smart functions.\n7.  **EchoStone:** A bit mystic, suggesting natural integration and responsiveness.\n8.  **Nimbus Living:** "Nimbus" refers to a clo

# 1) Plain Python (no LangChain magic)

In [15]:
def tool_search(query: str) -> str:
    return "25"  # pretend DB search returns "25 customers"

def tool_discount(customer_count: str) -> str:
    count = int(customer_count)
    return f"Discount = ${count * 5}"

# tool1 -> tool2
out1 = tool_search("customers named Smith")
out2 = tool_discount(out1)

print(out2)

Discount = $125


# 2) LCEL Sequential chain (recommended mindset)

In [17]:
from langchain.tools import tool

@tool
def tool_search(query: str) -> str:
    """Simulate searching customers and return the number of matching customers as a string."""
    return "25"  # pretend DB search returns "25 customers"

@tool
def tool_discount(customer_count: str) -> str:
    """Given a customer count (string), compute and return the discount text."""
    return f"Discount = ${int(customer_count) * 5}"

# LCEL chain: tool_search → tool_discount
chain = tool_search | tool_discount

print(chain.invoke("customers named Smith"))

Discount = $125


In [18]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def tool_search(query: str) -> str:
    """Simulate searching customers and return the number of matching customers as a string."""
    return "25"  # pretend DB search returns "25 customers"

@tool
def tool_discount(customer_count: str) -> str:
    """Given a customer count (string), compute and return the discount text."""
    return f"Discount = ${int(customer_count) * 5}"

llm = ChatOpenAI(model="gpt-4o-mini")  # or reuse openai_llm from cell 1
tools = [tool_search, tool_discount]

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are a helpful assistant. "
        "First use tool_search to find how many customers match, "
        "then use tool_discount to compute the discount."
    ),
)

result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Find customers named Smith and compute the discount at $5 per customer.",
        }
    ]
})

print(result["messages"][-1].content)

There are 25 customers named Smith, and the total discount computed at $5 per customer is $125.


# 3) Tool1 returns JSON → Tool2 consumes JSON (most robust)

In [22]:
# 3) Tool1 returns JSON → Tool2 consumes JSON (most robust)

import json
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# ---- Pydantic input schemas ----

class SearchCustomersJsonArgs(BaseModel):
    query: str = Field(..., description="Customer search query, e.g. 'customers named Smith'")

class DiscountFromSearchJsonArgs(BaseModel):
    search_json: str = Field(
        ...,
        description="JSON string returned by search_customers_json (with customer_count and query).",
    )

# ---- Tools using args_schema ----

@tool("search_customers_json", args_schema=SearchCustomersJsonArgs)
def search_customers_json(query: str) -> str:
    """Search customers and return JSON with customer_count and query."""
    data = {"customer_count": 25, "query": query}
    return json.dumps(data)

@tool("discount_from_search_json", args_schema=DiscountFromSearchJsonArgs)
def discount_from_search_json(search_json: str) -> str:
    """Take JSON from search_customers_json and return JSON with discount."""
    data = json.loads(search_json)
    count = data["customer_count"]
    return json.dumps(
        {
            "query": data["query"],
            "customer_count": count,
            "discount": count * 5,
        }
    )

llm = ChatOpenAI(model="gpt-4o-mini")  # or reuse openai_llm from cell 1

agent = create_agent(
    model=llm,
    tools=[search_customers_json, discount_from_search_json],
    system_prompt=(
        "You are a helpful assistant. "
        "First call search_customers_json to get customer_count as JSON, "
        "then call discount_from_search_json with that JSON to compute the discount. "
        "Explain the result clearly to the user."
    ),
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "For customers named Smith, compute the total discount at $5 per customer.",
            }
        ]
    }
)

print(result["messages"][-1].content)

I found a total of 25 customers named Smith. Since the discount is calculated at $5 per customer, the total discount amounts to $125.
